# Imports

In [18]:
import numpy as np
import pandas as pd



from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout
from tensorflow.keras.models import Sequential

# Data pre-processing

Due to the large size of dataset, many columns that are not needed and some rows containing null values have already been dropped

In [27]:
data = pd.read_csv("dataset.csv")

data = data.dropna()

print(data.shape)

print(data.info)

(131124, 18)
<bound method DataFrame.info of        neo pha      H  diameter  albedo  epoch_cal         e         a  \
0        N   N   3.40   939.400  0.0900   20190427  0.076009  2.769165   
1        N   N   4.20   545.000  0.1010   20200531  0.229972  2.773841   
2        N   N   5.33   246.596  0.2140   20200531  0.256936  2.668285   
3        N   N   3.00   525.400  0.4228   20190427  0.088721  2.361418   
4        N   N   6.90   106.699  0.2740   20200531  0.190913  2.574037   
...     ..  ..    ...       ...     ...        ...       ...       ...   
573686   N   N  15.60     4.491  0.0240   20200531  0.182925  2.937086   
573687   N   N  15.30     4.595  0.0920   20200531  0.073698  3.176190   
573688   N   N  15.40     4.086  0.1160   20200531  0.083151  3.229042   
573689   N   N  16.40     3.269  0.1040   20200531  0.210836  3.170190   
573690   N   N  16.80     2.132  0.0620   20200531  0.229331  2.920677   

               q          i          om           w        ad     

In [28]:
data

,neo,pha,H,diameter,albedo,epoch_cal,e,a,q,i,om,w,ad,n,tp_cal,per,moid,class
0,N,N,3.40,939.400,0.0900,20190427,0.076009,2.769165,2.558684,10.594067,80.305531,73.597695,2.979647,0.213885,20180430.25,1683.145703,1.59478,MBA
1,N,N,4.20,545.000,0.1010,20200531,0.229972,2.773841,2.135935,34.832932,173.024741,310.202392,3.411748,0.213345,20180721.46,1687.410992,1.23429,MBA
2,N,N,5.33,246.596,0.2140,20200531,0.256936,2.668285,1.982706,12.991043,169.851483,248.066193,3.353865,0.226129,20181123.29,1592.013769,1.03429,MBA
3,N,N,3.00,525.400,0.4228,20190427,0.088721,2.361418,2.151909,7.141771,103.810804,150.728541,2.570926,0.271609,20180509.06,1325.432763,1.13948,MBA
4,N,N,6.90,106.699,0.2740,20200531,0.190913,2.574037,2.082619,5.367427,141.571026,358.648418,3.065455,0.238661,20200317.22,1508.414421,1.09575,MBA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573686,N,N,15.60,4.491,0.0240,20200531,0.182925,2.937086,2.399819,27.965560,171.988705,161.151266,3.474352,0.195807,20180122.67,1838.541007,1.40243,MBA
573687,N,N,15.30,4.595,0.0920,20200531,0.073698,3.176190,2.942112,23.193462,43.051531,207.110303,3.410268,0.174118,20220911.64,2067.560001,1.94299,MBA
573688,N,N,15.40,4.086,0.1160,20200531,0.083151,3.229042,2.960545,16.428329,63.037073,72.835544,3.497539,0.169861,20210320.91,2119.380839,2.02639,OMB
573689,N,N,16.40,3.269,0.1040,20200531,0.210836,3.170190,2.501800,13.479527,80.175164,75.253005,3.838581,0.174613,20210522.88,2061.704725,1.55350,MBA


Training parameters


In [23]:
INPUT_SHAPE = (1,18)
OPTIMIZER = 'adam'
LOSS = 'binary_crossentropy'
METRICS = 'accuracy'

BATCH_SIZE = 128
N_EPOCHS = 20
VERBOSE = 1

# Generator model

In [ ]:
def make_generator():
  model = Sequential()
  
  model.add(Dense())
  model.add(BatchNormalization())



  return model

# Discriminator model
This is a binary CNN classifier, to determine whether the input is 'real' or not

In [24]:
def make_discriminator(INPUT_SHAPE):
  model = Sequential()

  model.add(Conv1D(64, kernel_size=5, padding='same', input_shape=INPUT_SHAPE))


  model.add(Dense(1, activation='sigmoid'))



  return model

In [25]:
model = make_discriminator(INPUT_SHAPE)

model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=METRICS)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 1, 64)             5824      
                                                                 
 dense_1 (Dense)             (None, 1, 1)              65        
                                                                 
Total params: 5,889
Trainable params: 5,889
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(data, batch_size=BATCH_SIZE, epochs=N_EPOCHS, verbose=VERBOSE)

ValueError: ignored